In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.backends.backend_pdf import PdfPages

name = 'point_'
output = 'fitted_parameters_'
N = 25 #number of measurements +1
Max = 2900 #maximum frequency of fitting peak
Min = 2500 #minimum frequency of fitting peak
initial_conditions = [0.01, 400, 22987.16, 50, 2680]
i = 0

def double_Lorentz(x, a, b, A, w, x_0, A1, w1, x_01):
    return a*x+b+(2*A/np.pi)*(w/(4*(x-x_0)**2 + w**2))+(2*A1/np.pi)*(w1/(4*(x-x_01)**2 + w1**2))

def Lorentz(x, a, b, A, w, x_0):
    return a*x+b+(2*A/np.pi)*(w/(4*(x-x_0)**2 + w**2))

for m in range(0,N):
    dane = np.loadtxt(name+str(m)+'.txt')
    time = dane[:,0]
    wave = dane[:,1]
    intensity = dane[:,2]

    lista_waves, lista_intensities = [], []
    begin_current_series = 0
    wave_diff_bool = (time[1:] != time[:-1])

    beginning_indices = np.where(wave_diff_bool)
    beginning_indices = np.append(0, beginning_indices) #zero index
    beginning_indices = np.append(beginning_indices, len(wave)) #1st index

    for i in range(len(beginning_indices[:-1])):
        beginning = beginning_indices[i]
        ending = beginning_indices[i+1]
        if i==0: 
            current_wave = wave[beginning : ending+1]
            current_intensity = intensity[beginning : ending+1]
        else:
            current_wave = wave[beginning+1 : ending+1]
            current_intensity = intensity[beginning+1 : ending+1]

        lista_waves.append(current_wave)
        lista_intensities.append(current_intensity)

    n = 0
    index_peak_end=0 #choosing a peak - here 2D peak of carbon nanotubes
    index_peak_begin=0
    for i in range(len(lista_waves)):
        while lista_waves[i][n] > Max:
            n = n+1
            index_peak_begin=n
        while lista_waves[i][n] > Min:
            n = n+1
            index_peak_end=n
        n=1

    for i in range(len(lista_waves)):
        intens_plot =lista_intensities[i][index_peak_begin:index_peak_end]
        wave_plot =lista_waves[i][index_peak_begin:index_peak_end]

    plt.plot(wave_plot, intens_plot) 
    
    x_zeros, w_fit, A_fit = [], [], []

    for i in range(len(lista_waves)): #here begins fitting Lorentian function to the chosen peak from the spectrum
        intens_plot =lista_intensities[i][index_peak_begin:index_peak_end]
        wave_plot =lista_waves[i][index_peak_begin:index_peak_end]
        fit_parameters, covariances = curve_fit(Lorentz, wave_plot, intens_plot, initial_conditions)
        a, b, A, w, x_0 = parametry
        x_fit = np.linspace(wave_plot.min(), wave_plot.max(), 1000)
        y_fit = Lorentz(x_fit, a, b, A, w, x_0)
        x_zeros.append(x_0)
        w_fit.append(w)
        A_fit.append(A)
    
    plt.plot(x_fit, y_fit, wave_plot, intens_plot )#will show the number of plots given as the number of read filed
    plt.show()                                     #helpful to see how fitting goes but time consuming way to do it

    powdata=open(output+str(m)+".txt","w")
    for val in zip(x_zeros, w_fit, A_fit):
        powdata.write('{} \t {} \t {}\n'.format(val[0], val[1], val[2]))
    powdata.close()